In [18]:
from SharedFolder import DuplicateFileorFolder
# Create Backup of Github_Repo
DuplicateFileorFolder('/Users/derekdewald/Documents/Python/Github_Repo/',
                      '/Users/derekdewald/Documents/GitHub_Repo_BACKUP/')

 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/61
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/0d
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/95
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/59
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/92
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/0c
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/66
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/3e
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/.git/objects/50
 Skipping `.git` folder: /Users/der

 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/Time-Series-Analysis-with-Python-Cookbook-Second-Edition/.git
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/Time-Series-Analysis-with-Python-Cookbook-Second-Edition/.git/objects
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/Time-Series-Analysis-with-Python-Cookbook-Second-Edition/.git/objects/pack
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/Time-Series-Analysis-with-Python-Cookbook-Second-Edition/.git/objects/info
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/Time-Series-Analysis-with-Python-Cookbook-Second-Edition/.git/info
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/Time-Series-Analysis-with-Python-Cookbook-Second-Edition/.git/logs
 Skipping `.git` folder: /Users/derekdewald/Documents/Python/Github_Repo/Time-Series-Analysis-with-Python-Cookbook-Second-Edition/.git/logs/refs
 

In [ ]:


def create_heatmap(df,column_name='',corr_value=.1,figsize=(20,15)):
    
    sns.set(style='white')
    
    # View column with Abbreviated title or full. Abbreviated displays nicer.
    corr = df.corr()
    
    if len(column_name)!=0:
        corr = corr[[column_name]]
        corr = corr[abs(corr[column_name])>corr_value]
    
    mask= np.zeros_like(corr,dtype=bool)
    mask[np.triu_indices_from(mask)]=True
    f,ax = plt.subplots(figsize=figsize)
    cmap = sns.diverging_palette(220,10,as_cmap=True)
    sns.heatmap(corr,mask=mask,cmap=cmap,vmax=.3,center=0,square=True,linewidths=.5)
    
    plt.title('Heat Map of Correlation')
    plt.show()
    
    
def create_column_inclusion_review(df,
                               columns,
                               column_type_df='',
                               decile_review_df=''):
    
    df = df[columns].copy()
    
    temp_df = review_dataset_dict(df)
    
    if len(column_type_df)!=0:
        temp_df = temp_df.merge(column_type_df,on='Financial Ratio',how='left')
    if len(decile_review_df)!=0:
        temp_df = temp_df.merge(decile_review_df,on='Financial Ratio',how='left')
    
    return temp_df


def variable_review(df,
                    column_name,
                    og_column,
                    column_inclusion_review_df,
                    corr_weight=.15):
        
    print(column_inclusion_review_df[column_inclusion_review_df['Financial Ratio']==column_name].T)
    print(f"\n")
    
    create_heatmap(df[og_columns],column_name)
    create_decile(df,column_name)
    
    print('Top 20 Records')
    print(df.sort_values(column_name)[[column_name,'BANKRUPTCY_FLAG']].tail(20))
    
    print('Bottom 20 Records')
    print(df.sort_values(column_name)[[column_name,'BANKRUPTCY_FLAG']].head(20))
    
    return df[df[column_name].isnull()].T

# Process for reviewing Non Tier 1 Elements.

def review_single_variable_manully(df,
                                   column_name,
                                   baseline_columns,
                                   column_inclusion_review_df,
                                   og_columns,
                                   export_to_excel=0):
        
    import datetime
    now = datetime.datetime.now()
    
    # Currently Included Columns for Simple Reference
    print("Columns Currently In Scope:")
    for included in baseline_columns:
        print(f"{included}\n")
    
    # EDA
    variable_review(df,column_name,og_columns,column_inclusion_review_df)    
    print(f"Number of Null Records with Bankrupcy Flag Yes: {df[df[column_name].isnull()]['BANKRUPTCY_FLAG'].sum()}")
    
    # Review Questions
    blank_or_remove = input('Remove Null Records/ Zero Null Records/ Exit Loop (remove/zero/exit)')
    include_in_model = input('Subjective Belief as to whether variable should be included in model (include/exclude)')
    negative_value = input('Remove, Zero or Leave Negative Values (remove/zero/ignore)')
    decision_logic = input('Please Provide Comment on Decision for Archival Reference')
        
    record_df =  pd.DataFrame([blank_or_remove.lower(),include_in_model.lower(),negative_value.lower(),decision_logic.lower(),now],index=['Null Record Approach','Baseline V2 Model Inclusion',"Negative Valuation",'Archival Decisioning','Extract Time'],columns=[column_name]).T.reset_index().rename(columns={'index':'Financial Ratio'})
    
    if export_to_excel==1:
        clean_column_name = clean_string(column_name,remove_chars=['+','-',"(",")",'/','*']).replace(" ","_")
        record_df.to_excel(f"manual_review/manual_review_{clean_column_name}_{now.strftime('%d%m%y%h%m%s')}.xlsx",index=False)
            
    return record_df

def read_files_in_folder(folder_location,file_type='*',import_df=0):
    
    files_ = os.listdir(folder_location)
    
    if file_type =='*':
        files_desired = files_.copy()
    else:
        files_desired = [x for x in files_ if x.find('xlsx')>-1]
        
    if import_df ==1:
        final_df = pd.DataFrame()
        if file_type =='xlsx':
            pd_read = pd.read_excel
        elif file_type =='csv':
            pd_read = pd.read_csv      
        else:
            print('Update Function')              
        for file in files_desired:
            final_df = pd.concat([final_df,pd_read(f"{folder_location}/{file}")])
            
        return final_df
         
    return files_desired



def build_binary_classification_model(input_dim, 
                                      hidden_layer_sizes,
                                      activation, 
                                      optimizer,
                                      learning_rate,
                                      metrics):

    """Build a binary classification model using Keras.

      Args:
        input_dim: Number of features in the input data.
        hidden_layer_sizes: A list with the number of units in each hidden layer.
        activation: The activation function to use for the hidden layers.
        optimizer: The optimizer
        learning_rate: The desired learning rate for the optimizer.

      Returns:
        model: A tf.keras model.
    """
    # Instantiate Model
    model = keras.models.Sequential()

    # Add Input Layer
    model.add(layers.InputLayer(input_shape=(input_dim,)))

    # Add Hidden Layers
    for nodes in hidden_layer_sizes:
        model.add(layers.Dense(units=nodes, activation=activation))

    # Add Output Layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Configure optimizer and compile the model
    if optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)

    return model


def train_model(X,
                y,
                input_dim,
                metrics,
                hidden_layer_sizes,
                activation, 
                optimizer,
                learning_rate,
                batch_size,
                num_epochs,
                validation_split,
                verbose=0):
                       

    # Build the model.
    model = build_binary_classification_model(input_dim=input_dim,
                                              hidden_layer_sizes=hidden_layer_sizes,
                                              activation=activation, 
                                              optimizer=optimizer,
                                              learning_rate=learning_rate,
                                              metrics=metrics)
    
    print(model.summary())     
                        
    # Train the model.
    history = model.fit(x=X,
                        y=y,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_split=validation_split,
                        verbose=verbose)

    # Retrieve the training metrics (after each train epoch) and the final test
    # accuracy.
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    plt.figure(figsize=(15,5))
    plt.plot(train_accuracy, label='train_accuracy')
    plt.plot(val_accuracy, label='validation accuracy')
    plt.xticks(range(num_epochs))
    plt.xlabel('Train epochs')
    plt.ylim(0, 1)
    plt.legend()
    plt.show()
    
 
    return history,model

def create_balanced_dataset(X,y,observations=0,column_name='BANKRUPTCY_FLAG'):
    
    '''
    Function to take observations and labels, combine them together and then select a even numher of random examples
    
    X - X_Test or X_Training
    y - y_test or y_training
    observation - Number of records from both Binary On and Binary Off Column
    column_name - Name of Binary Column to filter
    
    
    Used Default Value for purposes of reducing typing in code and given function created for Project Exclusively.
    
    '''
    
    # If length of observations is not defined, create a even 50/ 50 dataset
    if observations == 0:
        observations = y[column_name].sum()
        
    if observations>y[column_name].sum():
        observations = y[column_name].sum()
        
    temp_df = pd.concat([X,y],axis=1).copy()
    
    df1 = temp_df[temp_df[column_name]==1].sample(observations).copy()
    df2 = temp_df[temp_df[column_name]==0].sample(observations).copy()
    
    final_df = pd.concat([df1,df2])
    final_df = final_df.sample(frac=1)
    
    X = final_df.drop(column_name,axis=1)
    y = final_df[[column_name]]
    
    
    def train_neural_network(X,
                y,
                input_dim,
                metrics,
                hidden_layer_sizes,
                activation, 
                optimizer,
                learning_rate,
                batch_size,
                num_epochs,
                validation_split,
                verbose=0):
                       

    # Build the model.
    model = build_binary_classification_model(input_dim=input_dim,
                                              hidden_layer_sizes=hidden_layer_sizes,
                                              activation=activation, 
                                              optimizer=optimizer,
                                              learning_rate=learning_rate,
                                              metrics=metrics)
    
    print(model.summary())     
                        
    # Train the model.
    history = model.fit(x=X,
                        y=y,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_split=validation_split,
                        verbose=verbose)

    # Retrieve the training metrics (after each train epoch) and the final test
    # accuracy.
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    plt.figure(figsize=(15,5))
    plt.plot(train_accuracy, label='train_accuracy')
    plt.plot(val_accuracy, label='validation accuracy')
    plt.xticks(range(num_epochs))
    plt.xlabel('Train epochs')
    plt.ylim(0, 1)
    plt.legend()
    plt.show()
    
 
    return history,model





def knn_model_creation(X_df,
                       y_df,
                       neighbors=3):    
    model = KNeighborsClassifier(n_neighbors=neighbors)
    model.fit(X_df,y_df)    
    return model

def logistic_regression_creation(X_df,
                                 y_df,
                                 solver='liblinear'):
    '''
    ['liblinear','newton-cg','lbfgs','sag','saga']
    
    for i in [10,20,30]:
        temp_df = (y_probabilities >= i/100).astype(int)
        knn_df = pd.concat([knn_df,pd.DataFrame(temp_df,columns=[f'LR_{str(i)}p'])],axis=1)
        
        
    '''
    model = LogisticRegression(solver='liblinear')
    model.fit(X_df,y_df)
    return model


def random_forest_classifier(X_df,
                             y_df):
    '''
    important_features_rf = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False).head(10)
    
    
    '''
    model = RandomForestClassifier()
    model.fit(X_df,y_df)
    return model

def neural_network(X_df,
                   y_df,
                   input_dim, 
                   hidden_layer_sizes,
                   activation,
                   optimizer,
                   learning_rate,
                   metrics,
                   verbose=0):

    """Build a binary classification model using Keras.

      Args:
        input_dim: Number of features in the input data.
        hidden_layer_sizes: A list with the number of units in each hidden layer.
        activation: The activation function to use for the hidden layers.
        optimizer: The optimizer
        learning_rate: The desired learning rate for the optimizer.

      Returns:
        model: A tf.keras model.
    """
    # Instantiate Model
    model = keras.models.Sequential()

    # Add Input Layer
    model.add(layers.InputLayer(input_shape=(input_dim,)))

    # Add Hidden Layers
    for nodes in hidden_layer_sizes:
        model.add(layers.Dense(units=nodes, activation=activation))

    # Add Output Layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Configure optimizer and compile the model
    if optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)

    return model
34/22:
    # Where am I going to take out the Validation Data.
    # Where am I going to do the Standardization of Data?
    # Where am I going to take out the testing data.



    Takes a Dataframe, which includes Training Data (which includes Label), 
    

    # Step 1 Scale Data
if scaling_method.lower() == 'standard_scalar':
    X = apply_standard_scaler(X_df,X_df.columns.values)
elif scaling_method.lower()=='min_max':
    X = apply_min_max_scaler(X_df,X_df.columns.values)

    if bankrupcy_observations == 'entire_dataset':
a        pass
    elif bankrupcy_observations == 'all_bankrupcies':
        X,y = create_balanced_dataset(X,y)
    else:
        X,y = create_balanced_dataset(X,y,bankrupcy_observations)
34/23:
def create_model(X_df,
                 y_df,model,
                 scaling_method = 'standard_scalar',
                 learning_rate=.05,
                 model_parameters={}):
    '''
    
    Function to generate ML Model, based on model type as defined in Parameters.
    Below values for illustration purposes, can utilize dictionary to update as desired.

    if model is neural_network, 
    model_parameters = {'hidden_layer_sizes':[8,16,32,64],
                                          'activation':'relu',
                                          'optimizer':'adam',
                                          'learning_rate':.05,
                                          'metrics':['accuracy', keras.metrics.Precision(),keras.metrics.Recall()]}
    if model Logistic Regression,
    model_parameters = {'solver':'liblinear'}

    if model knn,
    model_parameters = {'n_neighbors':3}

    if model random_forsest
    model_parameters = {}

    '''

    if model == 'neural_network':
        model = neural_network(X_df,
                               y_df,
                               input_dim=len(X.columns.values), 
                               hidden_layer_sizes= model_parameters['hidden_layer_sizes'],
                               activation=model_parameters['activation'], 
                               optimizer=model_parameters['optimizer'],
                               learning_rate=model_parameters['learning_rate'],
                               metrics=model_parameters['metrics'])

    elif model == 'logistic_regression':
        model = logistic_regression_creation(X_df,
                                             y_df,
                                             solver=model_parameters['solver'])
    elif model == 'random_forest':
        model = random_forest_classifier(X_df,
                                         y_df)
    elif model == 'knn':
        model = knn_model_creation(X_df,
                                   y_df,
                                   neighbors=model_parameters['n_neighbors'])
    return model

def generate_predicition(X_test_df,
                         y_test_df,
                         model,
                         py_model,
                         model_parameters):

    if model == 'neural_network':
        history = py_model.fit(x=X_test_df,
                            y=y_test_df,
                            batch_size=model_parameters['batch_size'],
                            epochs=model_parameters['num_epochs'])

        acc = pd.DataFrame(history.history['accuracy'],columns=["Training Accuracy"])
        loss = pd.DataFrame(history.history['loss'],columns=["Training Loss"])
        prec_str = columns=[x for x in history.history.keys() if x.find('pre')!=-1][0]
        recall_str = columns=[x for x in history.history.keys() if x.find('reca')!=-1][0]
        precision = pd.DataFrame(history.history[prec_str],columns=[prec_str]).rename(columns={prec_str:'Precision'})
        recall = pd.DataFrame(history.history[recall_str],columns=[recall_str]).rename(columns={recall_str:'Recall'})

        model_results_df = pd.concat([model_results_df,pd.DataFrame([x for x in range(0,len(model_results_df))],columns=['Epoch Number'])],axis=1)
        model_results_df['Dataset'] = 'To Be Determined'
        model_results_df['Model'] = model
        model_results_df['activation'] = model_parameters['activation']
        model_results_df['optimizer'] = model_parameters['optimizer']
        model_results_df['learning_rate'] = model_parameters['learning_rate']
        model_results_df['batch_size'] = model_parameters['batch_size']
        model_results_df['epochs'] = model_parameters['num_epochs']
        model_results_df['hidden_layer_sizes'] = text_manipulation(model_parameters['hidden_layer_sizes'])
        #model_results_df['bankrupt_observations'] = bankrupcy_observations
        #model_results_df['total_observations_read'] = len(X)
        return model_results_df
    
    elif model == 'knn':
        pred = py_model.predict(X_test_df)
        pred_df = pd.DataFrame(pred,columns=['Predicitions'])
        final_df = pd.concat([y_test_df.reset_index(drop=True),pred_df],axis=1)
        return final_df
    
    else:
        pred = py_model.predict(X_test_df,y_test_df)
        pred_df = pd.DataFrame(pred,columns=['Predicitions'])
        final_df = pd.concat([y_test_df.reset_index(drop=True),pred_df],axis=1)
        return final_df
    

    # else:
    #     predicition = py_model.predict(X_test)
    #     temp_x = pd.DataFrame(predicition,columns=[column_name])
    #     if df == "":
    #         return pd.concat([y_test_df,temp_df],axis=1)
    #     else:
    #         return pd.concat([df,temp_df],axis=1)
34/24:
X = dataset_dictionary['All Ratios Cleaned']
y = dataset_dictionary['labels']

model='logistic_regression'
model_parameters = {'solver':'liblinear'}

lr_  = create_model(X,
                      y,
                      model=model,
                      model_parameters=model_parameters)

model = 'knn'
model_parameters = {'n_neighbors':3}

knn_ = create_model(X,
                      y,
                      model=model,
                      model_parameters=model_parameters)
34/25: knn_.predict(X)
34/26: X
34/27: X_train, X_test, y_train, y_test  = train_test_split(X,y,test_size=0.3, random_state=15)
34/28:
X = dataset_dictionary['All Ratios Cleaned']
y = dataset_dictionary['labels']

X_train, X_test, y_train, y_test  = train_test_split(X,y,test_size=0.3, random_state=15)


model='logistic_regression'
model_parameters = {'solver':'liblinear'}

lr_  = create_model(X_train,
                      y_train,
                      model=model,
                      model_parameters=model_parameters)

model = 'knn'
model_parameters = {'n_neighbors':3}

knn_ = create_model(X_train,
                      y_train,
                      model=model,
                      model_parameters=model_parameters)
34/29: lr_.n_iter_
34/30:
X = dataset_dictionary['All Ratios Cleaned']
y = dataset_dictionary['labels']

X_train, X_test, y_train, y_test  = train_test_split(X,y,test_size=0.3, random_state=15)


model='logistic_regression'
model_parameters = {'solver':'lbfgs'}

lr_  = create_model(X_train,
                      y_train,
                      model=model,
                      model_parameters=model_parameters)

model = 'knn'
model_parameters = {'n_neighbors':3}

knn_ = create_model(X_train,
                      y_train,
                      model=model,
                      model_parameters=model_parameters)
34/31:


def knn_model_creation(X_df,
                       y_df,
                       neighbors=3):    
    model = KNeighborsClassifier(n_neighbors=neighbors)
    model.fit(X_df,y_df)    
    return model

def logistic_regression_creation(X_df,
                                 y_df,
                                 max_iter=100,
                                 solver='liblinear'):
    '''
    ['liblinear','newton-cg','lbfgs','sag','saga']
    
    for i in [10,20,30]:
        temp_df = (y_probabilities >= i/100).astype(int)
        knn_df = pd.concat([knn_df,pd.DataFrame(temp_df,columns=[f'LR_{str(i)}p'])],axis=1)
        
        
    '''
    model = LogisticRegression(solver='liblinear',max_iter=max_iter)
    model.fit(X_df,y_df)
    return model


def random_forest_classifier(X_df,
                             y_df):
    '''
    important_features_rf = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False).head(10)
    
    
    '''
    model = RandomForestClassifier()
    model.fit(X_df,y_df)
    return model

def neural_network(X_df,
                   y_df,
                   input_dim, 
                   hidden_layer_sizes,
                   activation,
                   optimizer,
                   learning_rate,
                   metrics,
                   verbose=0):

    """Build a binary classification model using Keras.

      Args:
        input_dim: Number of features in the input data.
        hidden_layer_sizes: A list with the number of units in each hidden layer.
        activation: The activation function to use for the hidden layers.
        optimizer: The optimizer
        learning_rate: The desired learning rate for the optimizer.

      Returns:
        model: A tf.keras model.
    """
    # Instantiate Model
    model = keras.models.Sequential()

    # Add Input Layer
    model.add(layers.InputLayer(input_shape=(input_dim,)))

    # Add Hidden Layers
    for nodes in hidden_layer_sizes:
        model.add(layers.Dense(units=nodes, activation=activation))

    # Add Output Layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Configure optimizer and compile the model
    if optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)

    return model
34/32:
def create_model(X_df,
                 y_df,model,
                 scaling_method = 'standard_scalar',
                 learning_rate=.05,
                 model_parameters={}):
    '''
    
    Function to generate ML Model, based on model type as defined in Parameters.
    Below values for illustration purposes, can utilize dictionary to update as desired.

    if model is neural_network, 
    model_parameters = {'hidden_layer_sizes':[8,16,32,64],
                                          'activation':'relu',
                                          'optimizer':'adam',
                                          'learning_rate':.05,
                                          'metrics':['accuracy', keras.metrics.Precision(),keras.metrics.Recall()]}
    if model Logistic Regression,
    model_parameters = {'solver':'liblinear','max_iter':200}

    if model knn,
    model_parameters = {'n_neighbors':3}

    if model random_forsest
    model_parameters = {}

    '''

    if model == 'neural_network':
        model = neural_network(X_df,
                               y_df,
                               input_dim=len(X.columns.values), 
                               hidden_layer_sizes= model_parameters['hidden_layer_sizes'],
                               activation=model_parameters['activation'], 
                               optimizer=model_parameters['optimizer'],
                               learning_rate=model_parameters['learning_rate'],
                               metrics=model_parameters['metrics'])

    elif model == 'logistic_regression':
        model = logistic_regression_creation(X_df,
                                             y_df,
                                             max_iter=model_parameters['max_iter'],
                                             solver=model_parameters['solver'])
    elif model == 'random_forest':
        model = random_forest_classifier(X_df,
                                         y_df)
    elif model == 'knn':
        model = knn_model_creation(X_df,
                                   y_df,
                                   neighbors=model_parameters['n_neighbors'])
    return model
    

In [ ]:

RE has issue with Python escape clauses \, utilize r to make it string literal
r"\n"

* \b: Word boundary ensures that we match whole words.
* \w+: Matches one or more word characters (letters, digits, or underscores).
* \b: Another word boundary to ensure the end of the word.
* [^\w\s]: Matches any character that is not a word character (letters, digits, or underscores) or a whitespace character.
* |: OR operator.
* [\d]: Matches any digit.
* [^\w\s]: Matches any character that is not a word character (letters, digits, or underscores) or a whitespace character.
* |: OR operator.
* [\d]: Matches any digit.




Linux
Operating Systems are Layered.
Kernel is the inter most layer
Outermost layer is a GUI
Linux can peel off layers, helps towards simplicity
GUI can take  ~50% of processing power

Linux Distributions
GNU (GNU not Unix)
- Red Hat (CentOS)
- Fedora
Debian
- Ubuntu
- Raspberry Pi
- Various IoT
- 

Resources: 
- Command Line Help: www.explainshell.com


Linux Commands
man - Manual
History - history
Clear - clears history
Q - quits from current query
pwd- current directory
~ - short hand for home
. Current Directory
.. Parent Directory
Ls -l adds 6 additional fields 
ls -lh - adds file sizes
First letter - D: Directory, -: File, l - Link, Permission, Number of Hard links, Users who owns it. Size in bytes, date opened or last maintained  
Ls -la - shows hidden files
sudo = "super user (root) do"
mkdir - makes new directory
rmdir - removes directory
rm -r temp_1_3 - recursively removes directories
chmod 765 temp_1_1 - change permissions to directory


* r = read = 2^2 = 4
* w = write = 2^1 = 2
* x = execute = 2^0 = 1
* 1 = --x
* 2 = -w-
* 3 = -wx
* 4 = r--
* 5 = r-x
* 6 = rw-
* 7 = rwx

* u = user
* g = group
* o = other (warning: NOT owner!)

chmod u=rwx,g=rx,o=r *

ls -lhR - recursively list out directory tree
cp filename - copy
rm filename - remove
mv filename new_filename- rename

head returns the first few lines of a file; optionally pass the number of lines to return
tail returns the last few lines of a file; optionally pass the number of lines to return
grep matches a pattern in a file and returns lines that match the pattern
grep -v returns the opposite, the rows that do NOT match
| is a pipeline; Linux spawn a process for each command; Linux creates an "anonymous pipe" aka "un-named pipe" between the processes; Linux routes standard output to pipes and routes standard input from pipes for intermediate commands
diff finds the differences between two files
* vi filename
* Movement mode: arrow keys, page up, page down, 0 beginning of line, $ end of line, x delete character, dd delete line
* Movement mode to insert mode: i for insert, a for append, note that INSERT appears lower left
* Insert mode to movement mode: ESC key
* Movement mode to command mode: : note that hitting the enter key will run the command and return to movement mode
* Save changes: :w
* Save a file and exit: :wq
* Quit: :q
* Quit with unsaved changes: :q!


How to Change to Bash
chsh -s /bin/bash